# Synergy Logistics

**Synergy Logistics** es una empresa dedicada a la intermediación de servicios de importación y exportación de diferentes productos. Actualmente la empresa cuenta con una base de datos que refleja las rutas más importantes que opera desde el año 2015, con su respectivo origen y destino, año, producto, modo de transporte y valor total. Nuestro propósito, es que a partir de estos datos se genere un análisis que sirva de la base para la estructuración de su estrategia operativa.

Los datos que nos fueron proporcionados por Synergy Logistics son los siguientes.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("synergy_logistics_database.csv")
df.head(1).transpose().reset_index().rename(columns={'index': 'Nombre de Columna', 0: 'Ejemplo'})

,Nombre de Columna,Ejemplo
0,register_id,1
1,direction,Exports
2,origin,Japan
3,destination,China
4,year,2015
5,date,31/01/15
6,product,Cars
7,transport_mode,Sea
8,company_name,Honda
9,total_value,33000000


Con el fin de crear una estrategia operativa para el año 2021, tenemos analizar los datos proporcionados para obtener 3 enfoques:
- Rutas de importación y exportación
- Medios de transporte usados
- Valor de las importaciones y exportaciones por rutas.

## 10 rutas con más demandas

### Rutas de importaciones

In [2]:
rutas_importaciones = df.loc[df["direction"] == "Imports"]
rutas_importaciones = rutas_importaciones.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas_importaciones.reset_index()
rutas_importaciones.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas_importaciones.rename(columns={'total_value': 'count'}, inplace=True)
rutas_importaciones.sort_values('count', ascending=False).reset_index().head(10)

,direction,origin,destination,count
0,Imports,Singapore,Thailand,273
1,Imports,Germany,China,233
2,Imports,China,Japan,210
3,Imports,Japan,Mexico,206
4,Imports,China,Thailand,200
5,Imports,Malaysia,Thailand,195
6,Imports,Spain,Germany,142
7,Imports,Mexico,USA,122
8,Imports,China,United Arab Emirates,114
9,Imports,Brazil,China,113


### Rutas exportaciones

In [3]:
rutas_exportaciones = df.loc[df["direction"] == "Exports"]
rutas_exportaciones = rutas_exportaciones.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas_exportaciones.reset_index()
rutas_exportaciones.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas_exportaciones.rename(columns={'total_value': 'count'}, inplace=True)
rutas_exportaciones.sort_values('count', ascending=False).reset_index().head(10)

,direction,origin,destination,count
0,Exports,South Korea,Vietnam,497
1,Exports,Netherlands,Belgium,437
2,Exports,USA,Netherlands,436
3,Exports,China,Mexico,330
4,Exports,Japan,Brazil,306
5,Exports,Germany,France,299
6,Exports,South Korea,Japan,279
7,Exports,Australia,Singapore,273
8,Exports,Canada,Mexico,261
9,Exports,China,Spain,250


### Rutas importaciones y exportaciones

In [4]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).count()
rutas.reset_index()
rutas.drop(['register_id', 'year', 'date', 'product', 'transport_mode', 'company_name'], axis=1, inplace=True)
rutas.rename(columns={'total_value': 'count'}, inplace=True)
rutas.sort_values('count', ascending=False).reset_index().head(10)

,direction,origin,destination,count
0,Exports,South Korea,Vietnam,497
1,Exports,Netherlands,Belgium,437
2,Exports,USA,Netherlands,436
3,Exports,China,Mexico,330
4,Exports,Japan,Brazil,306
5,Exports,Germany,France,299
6,Exports,South Korea,Japan,279
7,Exports,Australia,Singapore,273
8,Imports,Singapore,Thailand,273
9,Exports,Canada,Mexico,261


Si comparamos esta tabla con la de la suma de los montos

In [5]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).sum()
rutas.reset_index()
rutas.drop(['register_id', 'year'], axis=1, inplace=True)
rutas.sort_values('total_value', ascending=False).reset_index().head(10)

,direction,origin,destination,total_value
0,Exports,China,Mexico,12250000000
1,Exports,Canada,Mexico,8450000000
2,Exports,South Korea,Vietnam,6877007000
3,Exports,France,Belgium,5538069000
4,Exports,France,United Kingdom,5427000000
5,Exports,China,South Korea,4790000000
6,Exports,USA,Mexico,4710000000
7,Exports,South Korea,Japan,4594000000
8,Exports,Germany,Italy,4541000000
9,Exports,China,Germany,4090000000


No se recomienda elegir las rutas con más demandas, ya que no son estas precisamente las que más monto generan.

## Medios de transporte más importantes

In [6]:
transportes = df.groupby(['transport_mode'], group_keys=False).sum()
transportes.drop(['register_id', 'year'], axis=1, inplace=True)
transportes.sort_values('total_value', ascending=False).reset_index()

,transport_mode,total_value
0,Sea,100530622000
1,Rail,43628043000
2,Air,38262147000
3,Road,33270486000


Los 3 medios de transporte más importantes para Synergy Logistics son: Mar, Ferrocarril y Aire.
El que genera menos valor es por Carretera, por lo que pueden reducir este.

Se recomienda encontrar maneras de completar las rutas por carretera a través de los otros medios de transporte para no perder totalmente las importaciones y exportaciones en esas rutas.

## 80% del valor de exportaciones y importaciones

In [7]:
rutas = df.groupby(['direction', 'origin', 'destination'], group_keys=False).sum()
gran_total = rutas['total_value'].sum()
rutas.reset_index()
rutas.drop(['register_id', 'year'], axis=1, inplace=True)
rutas.sort_values('total_value', ascending=False, inplace=True)

pd.set_option('display.float_format', '{:.10f}'.format)
rutas['porcentaje'] = rutas['total_value'].cumsum() / gran_total
rutas_80 = rutas.loc[rutas['porcentaje'] <= 0.8]
rutas_80.reset_index()

,direction,origin,destination,total_value,porcentaje
0,Exports,China,Mexico,12250000000,0.0567941318
1,Exports,Canada,Mexico,8450000000,0.0959704921
2,Exports,South Korea,Vietnam,6877007000,0.1278540546
3,Exports,France,Belgium,5538069000,0.1535299584
4,Exports,France,United Kingdom,5427000000,0.1786909178
...,...,...,...,...,...
56,Imports,Mexico,Japan,1143000000,0.7776517159
57,Imports,USA,India,1133000000,0.7829045936
58,Exports,Spain,Russia,1085000000,0.7879349310
59,Exports,India,United Arab Emirates,1037000000,0.7927427281


Elegimos solos los países de acuerdo al origin.

In [8]:
count_origin = rutas_80.groupby(['origin']).count().reset_index()
count_origin.drop(['total_value'], axis=1, inplace=True)
count_origin.rename(columns={'porcentaje': 'count'}, inplace=True)
count_origin

,origin,count
0,Belgium,1
1,Canada,1
2,China,11
3,France,5
4,Germany,5
5,India,1
6,Italy,1
7,Japan,7
8,Malaysia,1
9,Mexico,2


Veamos los países presentes en el 100% de las importaciones y exportaciones.

In [9]:
count_origin_100 = rutas.groupby(['origin']).count().reset_index()
count_origin_100.drop(['total_value'], axis=1, inplace=True)
count_origin_100.rename(columns={'porcentaje': 'count'}, inplace=True)
count_origin_100

,origin,count
0,Australia,7
1,Austria,6
2,Belgium,4
3,Brazil,7
4,Canada,7
5,China,16
6,France,13
7,Germany,13
8,India,6
9,Italy,14


Comparamos los países y vemos aquellos fuera del 80%, estos son los que podemos ignorar.

In [10]:
paises_descartados = []

list_100 = count_origin_100['origin'].tolist()
list_80 = count_origin['origin'].tolist()

for pais in list_100:
    if pais not in list_80:
        paises_descartados.append(pais)
        
paises_descartados

['Australia',
 'Austria',
 'Brazil',
 'Switzerland',
 'United Arab Emirates',
 'Vietnam']

## Conclusión general

A Synergy Logistics le conviene implementar dos estrategias: basada en las rutas que le generan el 80% del valor total y también quedarse con los 3 mejores medios de transportes.

En el primer caso, estaría ahorrando mucho dinero al quitar instalaciones que tiene en 6 paises.
Ya que tener presencia en países en los que no hay tanto valor no es tan rentable.
Se estaría enfocando en 17 de 23 paises en caso de seguir esta estrategía, lo cual son 6 paises menos.

En el caso de descartar el medio de transporte que menos valor genera, se tendría que consultar si es viable que los otros medios de transporte cubran estas rutas.
De lo contrario, tendríamos problemas porque el valor para el Carretera es casi el mismo que por Aire y estaríamos perdiendo mucho valores.

Lo que no se recomienda es seguir con la estrategia de usar el total de las exportaciones o importaciones.
Ya que solo unas cuantas de estas rutas están también adentro del top 10 de rutas que más valor tienen. 